В якості бази даних використовується PostgreSQL, налаштування знаходиться у [Docker compose файлі](./compose.yaml).
Проблем не виникло

In [1]:
import psycopg2

In [2]:
master_connection = psycopg2.connect(dbname="development_database", user="postgres",
                                     password='postgres', host='localhost', port=5432)
slave_connection = psycopg2.connect(dbname="development_database", user="repl_user",
                                    password='repl_user', host='localhost', port=5433)

Створюємо таблицю в мастер ноді та пишемо в неї тестові дані

In [3]:
with master_connection:
    with master_connection.cursor() as cursor:
        cursor.execute('DROP TABLE IF EXISTS messages')
        cursor.execute('CREATE TABLE IF NOT EXISTS messages (id serial, body varchar)')
        cursor.execute('INSERT INTO messages (body) values (%s)', ('Test message',))

Перевіряємо, що щойно записані дані можна вичитати зі slave ноди.

In [4]:
with slave_connection:
    with slave_connection.cursor() as cursor:
        cursor.execute('SELECT * FROM messages')
        print(cursor.fetchall())

[(1, 'Test message')]
